In [185]:
import pandas as pd # import library
import numpy as np # import lib
import statistics as st
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from numpy import linalg as LA
from sklearn import metrics

df=pd.read_csv(r'/home/tharaka/kagglecompetition/data-storm-20/Hotel-A-train.csv' )
dfval = pd.read_csv(r'/home/tharaka/kagglecompetition/data-storm-20/Hotel-A-validation.csv')
# train set

df['Expected_checkin'] = pd.to_datetime(df['Expected_checkin'],infer_datetime_format=True)
df['Expected_checkout'] = pd.to_datetime(df['Expected_checkout'],infer_datetime_format=True)
df['Booking_date'] = pd.to_datetime(df['Booking_date'],infer_datetime_format=True)

df['Days'] = df['Expected_checkin']-df['Booking_date']
df['Days'] = df.Days.dt.days
#dfdf2=df.

df = df[df.Days >0]

df = df.drop(['Gender','Ethnicity','Country_region','Hotel_Type','Reservation-id','Expected_checkin','Expected_checkout','Booking_date','Room_Rate'],axis=1)

dfval['Expected_checkin'] = pd.to_datetime(dfval['Expected_checkin'],infer_datetime_format=True)
dfval['Expected_checkout'] = pd.to_datetime(dfval['Expected_checkout'],infer_datetime_format=True)
dfval['Booking_date'] = pd.to_datetime(dfval['Booking_date'],infer_datetime_format=True)

dfval['Days'] = dfval['Expected_checkin']-dfval['Booking_date']
dfval['Days'] = dfval.Days.dt.days
dfval = dfval[dfval.Days >0]
dfval = dfval.drop(['Gender','Ethnicity','Country_region','Hotel_Type','Reservation-id','Expected_checkin','Expected_checkout','Booking_date','Room_Rate'],axis=1)

df.loc[df['Deposit_type']=='No Deposit','Deposit_type']=0
df.loc[df['Deposit_type']=='Refundable','Deposit_type']=1
df.loc[df['Deposit_type']=='Non-Refundable','Deposit_type']=2

df.loc[df['Income']=='<25K','Income']=0
df.loc[df['Income']=='25K --50K','Income']=1
df.loc[df['Income']=='50K -- 100K','Income']=2
df.loc[df['Income']=='>100K','Income']=3


dfval.loc[dfval['Deposit_type']=='No Deposit','Deposit_type']=0
dfval.loc[dfval['Deposit_type']=='Refundable','Deposit_type']=1
dfval.loc[dfval['Deposit_type']=='Non-Refundable','Deposit_type']=2

dfval.loc[dfval['Income']=='<25K','Income']=0
dfval.loc[dfval['Income']=='25K --50K','Income']=1
dfval.loc[dfval['Income']=='50K -- 100K','Income']=2
dfval.loc[dfval['Income']=='>100K','Income']=3
money = ['Deposit_type','Income']
dfval["Money"] = dfval[money].sum(axis=1)
df["Money"] = df[money].sum(axis=1)

X_train = df.copy()
y_train = df['Reservation_Status']
X_valid = dfval.copy()
y_valid = dfval.pop('Reservation_Status')



In [186]:
X_valid

,Age,Educational_Level,Income,Adults,Children,Babies,Meal_Type,Visted_Previously,Previous_Cancellations,Deposit_type,Booking_channel,Required_Car_Parking,Reservation_Status,Use_Promotion,Discount_Rate,Days,Money
0,56,Grad,0,2,2,0,HB,No,No,0,Agent,No,No-Show,Yes,15,195,0.0
1,60,College,1,2,2,0,FB,Yes,No,0,Online,Yes,Canceled,No,0,175,1.0
2,58,College,0,3,1,0,FB,No,No,0,Direct,No,Canceled,Yes,10,193,0.0
3,23,College,1,1,2,0,FB,Yes,No,1,Direct,No,Check-In,Yes,25,103,2.0
4,47,College,1,1,1,0,HB,Yes,No,0,Online,Yes,Check-In,Yes,10,92,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2744,22,College,2,5,1,1,BB,Yes,No,1,Online,No,Check-In,Yes,10,37,3.0
2745,35,College,1,3,2,0,FB,No,No,2,Direct,Yes,Canceled,Yes,5,154,3.0
2746,18,Grad,1,2,3,0,FB,No,No,1,Agent,No,No-Show,Yes,20,30,2.0
2747,42,College,0,5,1,0,HB,Yes,Yes,0,Online,Yes,Check-In,Yes,20,170,0.0


In [187]:
y_train.loc[y_train=='Check-In']=1
y_train.loc[y_train=='Canceled']=2
y_train.loc[y_train=='No-Show']=3
y_valid.loc[y_valid=='Check-In']=1
y_valid.loc[y_valid=='Canceled']=2
y_valid.loc[y_valid=='No-Show']=3


/home/tharaka/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [188]:
test=X_valid
#test['Reservation_Status']=y_valid
y_test=y_valid.astype(int)
train=X_train


train['Reservation_Status']=y_train.astype(int)

In [189]:
test

,Age,Educational_Level,Income,Adults,Children,Babies,Meal_Type,Visted_Previously,Previous_Cancellations,Deposit_type,Booking_channel,Required_Car_Parking,Reservation_Status,Use_Promotion,Discount_Rate,Days,Money
0,56,Grad,0,2,2,0,HB,No,No,0,Agent,No,No-Show,Yes,15,195,0.0
1,60,College,1,2,2,0,FB,Yes,No,0,Online,Yes,Canceled,No,0,175,1.0
2,58,College,0,3,1,0,FB,No,No,0,Direct,No,Canceled,Yes,10,193,0.0
3,23,College,1,1,2,0,FB,Yes,No,1,Direct,No,Check-In,Yes,25,103,2.0
4,47,College,1,1,1,0,HB,Yes,No,0,Online,Yes,Check-In,Yes,10,92,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2744,22,College,2,5,1,1,BB,Yes,No,1,Online,No,Check-In,Yes,10,37,3.0
2745,35,College,1,3,2,0,FB,No,No,2,Direct,Yes,Canceled,Yes,5,154,3.0
2746,18,Grad,1,2,3,0,FB,No,No,1,Agent,No,No-Show,Yes,20,30,2.0
2747,42,College,0,5,1,0,HB,Yes,Yes,0,Online,Yes,Check-In,Yes,20,170,0.0


In [190]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2723 entries, 0 to 2748
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Age                     2723 non-null   int64  
 1   Educational_Level       2723 non-null   object 
 2   Income                  2723 non-null   object 
 3   Adults                  2723 non-null   int64  
 4   Children                2723 non-null   int64  
 5   Babies                  2723 non-null   int64  
 6   Meal_Type               2723 non-null   object 
 7   Visted_Previously       2723 non-null   object 
 8   Previous_Cancellations  2723 non-null   object 
 9   Deposit_type            2723 non-null   object 
 10  Booking_channel         2723 non-null   object 
 11  Required_Car_Parking    2723 non-null   object 
 12  Reservation_Status      2723 non-null   object 
 13  Use_Promotion           2723 non-null   object 
 14  Discount_Rate           2723 non-null   

In [191]:
def encode_target_smooth(data, target, categ_variables, smooth):
    """    
    Apply target encoding with smoothing.
    
    Parameters
    ----------
    data: pd.DataFrame
    target: str, dependent variable
    categ_variables: list of str, variables to encode
    smooth: int, number of observations to weigh global average with
    
    Returns
    --------
    encoded_dataset: pd.DataFrame
    code_map: dict, mapping to be used on validation/test datasets 
    defaul_map: dict, mapping to replace previously unseen values with
    """
    train_target = data.copy()
    code_map = dict()    # stores mapping between original and encoded values
    default_map = dict() # stores global average of each variable
    
    for v in categ_variables:
        prior = data[target].mean()
        n = data.groupby(v).size()
        mu = data.groupby(v)[target].mean()
        mu_smoothed = (n * mu + smooth * prior) / (n + smooth)
        
        train_target.loc[:, v] = train_target[v].map(mu_smoothed)        
        code_map[v] = mu_smoothed
        default_map[v] = prior        
    return train_target, code_map, default_map


In [192]:
cat_vars = ['Educational_Level', 'Children', 'Babies', 'Meal_Type', 'Previous_Cancellations',
            'Use_Promotion', 'Required_Car_Parking','Money']

In [193]:
test=test.drop(['Reservation_Status','Deposit_type','Booking_channel','Visted_Previously','Days'],axis=1)
train=train.drop(['Deposit_type','Booking_channel','Visted_Previously','Days'],axis=1)

In [194]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2723 entries, 0 to 2748
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Age                     2723 non-null   int64  
 1   Educational_Level       2723 non-null   object 
 2   Income                  2723 non-null   object 
 3   Adults                  2723 non-null   int64  
 4   Children                2723 non-null   int64  
 5   Babies                  2723 non-null   int64  
 6   Meal_Type               2723 non-null   object 
 7   Previous_Cancellations  2723 non-null   object 
 8   Required_Car_Parking    2723 non-null   object 
 9   Use_Promotion           2723 non-null   object 
 10  Discount_Rate           2723 non-null   int64  
 11  Money                   2723 non-null   float64
dtypes: float64(1), int64(5), object(6)
memory usage: 276.6+ KB


In [195]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26820 entries, 0 to 27498
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Age                     26820 non-null  int64  
 1   Educational_Level       26820 non-null  object 
 2   Income                  26820 non-null  object 
 3   Adults                  26820 non-null  int64  
 4   Children                26820 non-null  int64  
 5   Babies                  26820 non-null  int64  
 6   Meal_Type               26820 non-null  object 
 7   Previous_Cancellations  26820 non-null  object 
 8   Required_Car_Parking    26820 non-null  object 
 9   Reservation_Status      26820 non-null  int64  
 10  Use_Promotion           26820 non-null  object 
 11  Discount_Rate           26820 non-null  int64  
 12  Money                   26820 non-null  float64
dtypes: float64(1), int64(6), object(6)
memory usage: 2.9+ MB


In [196]:
train_target_smooth, target_map, default_map = encode_target_smooth(train, 'Reservation_Status', cat_vars, 500)
test_target_smooth = test.copy()
for v in cat_vars:
    test_target_smooth.loc[:, v] = test_target_smooth[v].map(target_map[v])

In [197]:
train_target_smooth_x=train_target_smooth.drop(['Reservation_Status'],axis=1)

In [198]:
y_train2=y_train.astype(int)

In [415]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=1)
clf = clf.fit(train_target_smooth_x,y_train2)
y_pred = clf.predict(test_target_smooth)




from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred)) 

from sklearn.metrics import f1_score
print(f1_score(y_test, y_pred, average='macro'))

[[1213  248  135]
 [ 538  128   68]
 [ 275   70   48]]
              precision    recall  f1-score   support

           1       0.60      0.76      0.67      1596
           2       0.29      0.17      0.22       734
           3       0.19      0.12      0.15       393

    accuracy                           0.51      2723
   macro avg       0.36      0.35      0.35      2723
weighted avg       0.46      0.51      0.47      2723

0.3452710561703464


In [416]:
from sklearn.ensemble import AdaBoostClassifier
AdaBoost=AdaBoostClassifier(base_estimator=clf,n_estimators=1000,learning_rate=1)
boostmodel=AdaBoost.fit(train_target_smooth_x,y_train2)
y_pred=boostmodel.predict(test_target_smooth)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred)) 

from sklearn.metrics import f1_score
print(f1_score(y_test, y_pred, average='macro'))

[[1252  255   89]
 [ 579  115   40]
 [ 307   69   17]]
              precision    recall  f1-score   support

           1       0.59      0.78      0.67      1596
           2       0.26      0.16      0.20       734
           3       0.12      0.04      0.06       393

    accuracy                           0.51      2723
   macro avg       0.32      0.33      0.31      2723
weighted avg       0.43      0.51      0.46      2723

0.30991758180930584


In [236]:
dftest = pd.read_csv(r'/home/tharaka/kagglecompetition/data-storm-20/Hotel-A-test.csv')

In [237]:
dftest['Expected_checkin'] = pd.to_datetime(dftest['Expected_checkin'],infer_datetime_format=True)
dftest['Booking_date'] = pd.to_datetime(dftest['Booking_date'],infer_datetime_format=True)
dftest['Days'] = dftest['Expected_checkin']-dftest['Booking_date']
dftest['Days'] = dftest.Days.dt.days
#dfdf2=df.



dftest = dftest.drop(['Gender','Ethnicity','Country_region','Hotel_Type','Reservation-id','Expected_checkin','Expected_checkout','Booking_date','Room_Rate','Days'],axis=1)


dftest.loc[dftest['Deposit_type']=='No Deposit','Deposit_type']=0
dftest.loc[dftest['Deposit_type']=='Refundable','Deposit_type']=1
dftest.loc[dftest['Deposit_type']=='Non-Refundable','Deposit_type']=2

dftest.loc[dftest['Income']=='<25K','Income']=0
dftest.loc[dftest['Income']=='25K --50K','Income']=1
dftest.loc[dftest['Income']=='50K -- 100K','Income']=2
dftest.loc[dftest['Income']=='>100K','Income']=3


money = ['Deposit_type','Income']
dftest["Money"] = dftest[money].sum(axis=1)


X_test1 = dftest.copy()


In [238]:
X_test1=X_test1.drop(['Deposit_type','Booking_channel','Visted_Previously'],axis=1)

In [239]:
test_target_smooth1 = X_test1.copy()
for v in cat_vars:
    test_target_smooth1.loc[:, v] = test_target_smooth1[v].map(target_map[v])

In [240]:
y_pred2 = clf.predict(test_target_smooth1)
y_pred2


array([1, 3, 2, ..., 1, 1, 1])

In [241]:
lasted=pd.DataFrame(y_pred2)

In [242]:
lasted

,0
0,1
1,3
2,2
3,1
4,1
...,...
4313,2
4314,1
4315,1
4316,1


In [244]:
lasted.to_csv('finalist.csv')